In [4]:
import gym
import pygame

In [5]:
pygame.init()
pygame.display.list_modes()

[(2880, 1800),
 (2560, 1600),
 (2048, 1280),
 (1920, 1200),
 (1680, 1050),
 (1440, 900),
 (1280, 800),
 (1024, 768),
 (1024, 640),
 (840, 525),
 (800, 600),
 (720, 450),
 (640, 480)]

In [5]:
env = gym.make('CartPole-v1')
state = env.reset()
for t in range(100):
    env.render()
    action = env.action_space.sample() # random action
    state, reward, done, info = env.step(action)
    if done:
        print("Episode finished in {} steps".format(t))
        break
env.close()

Episode finished in 30 steps


In [3]:
env = gym.make("FrozenLake-v1",is_slippery=False)

In [11]:
env.observation_space

Discrete(16)

In [10]:
env.observation_space.n

16

In [12]:
#sample from the state space
for _ in range(10):
    print(env.observation_space.sample())

11
12
1
4
6
6
1
4
1
5


In [13]:
env.action_space

Discrete(4)

In [16]:
#sample from the state space
for _ in range(10):
    print(env.action_space.sample())

3
3
2
2
0
1
2
1
2
2


In [4]:
#visualize env
env.reset()
env.render()

In [ ]:
#test action 1
env.reset()
action = 1
for _ in range(5):
    env.render()
    state, reward, done, info = env.step(action)

In [ ]:
# hard coded trajectory
env.reset()
optimal_actions = [1,1,2,1,2,2]
for t in range(len(optimal_actions)):
    env.render()
    action = optimal_actions[t]
    state, reward, done, info = env.step(action)
    print('s = {}, r = {}, d = {}'.format(state, reward, done))
env.render()


In [ ]:
# Policy iteration
######

In [3]:
import numpy as np
# import utils
import solutions


ModuleNotFoundError: No module named 'solutions'

In [6]:
env = gym.make('FrozenLake-v1',is_slippery=False)

num_steps = env._max_episode_steps
num_states = env.oservation_space.n
num_actions = env.action_space.n
env_model = env.P
gamma = 1.0

In [ ]:
#policy iteration
# loop thorough all of our actions through all possible next states...


In [7]:
# test with a random policy
def random_policy(num_states, num_actions):
    return np.ones((num_states,num_actions)) / num_actions

In [1]:
utilsb

NameError: name 'utils' is not defined

In [ ]:
# Q-Learning
# no access to environmental model (probabilities)
# must use noise or exlporation and iterative calculation of probability
num_training_episodes = 5000
num_eval_episodes = 50
num_steps = env._max_episode_steps
num_states = env.observation_sapce.n
num_actions = env.action_space.n

gamma = 1
alpha = .1
max_epsilon = 1
min_epsilon = .01
epsilon_decay = .005
eval_itrs = 50


In [ ]:
# Initialize Q(s,a) for all s _E_ S and a _E_ A
# For each Episode:
#     reset evnireonment, roll out trajectory?
#     select action a_t for state s_t (policy derived from (s,a)).
#     take action a_t, observe reward r_t+1 and next state s_t+1.
#     Update Q(s_t,a_t) := Q(s_t,a_t) + alpha[r_t+1 + gamma* max(a)Q(s_t+1, a)- Q(s_t,a_t)]


In [ ]:
# Assignment
# Definitions 
# 8x8 environment, totalling 64 states, 
# all states have reward of 0, except (7,7)
# the bottom right. 
# 
# policy iteration v policy evaluation
# 
# policy is state x action matrix offf probabilities?
# training loop
def evaluate_policy(policy, num_episodes=10, render=False):
    returns = list()
    for e in range(num_episodes):
        total_reward = 0
        done = False
        state = env.reset()
        
        t = 0 
        while not done:
            if render:
                env.render()
                
            # takes best action
            action = np.argmax(policy[state])
            state, reward, done, _ = env.step(action)
            
            total_reward += reward
            if done:
                returns.append(total_reward)
                break
                
    return returns 

In [ ]:
def initiate_policy():
    # initiate random policy
    return policy

In [26]:
# Policy Evaluation
# def policy_evaluation(s, t):
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
        # Number of evaluation iterations
        evaluation_iterations = 1
        # Initialize a value function for each state as zero
        V = np.zeros(environment.observation_space.n)
        # Repeat until change in value is below the threshold
        for i in range(int(max_iterations)):
                # Initialize a change of value function as zero
                delta = 0
                # Iterate though each state
                for state in range(environment.observation_space.n):
                       # Initial a new value of current state
                       v = 0
                       # Try all possible actions which can be taken from this state
                       for action, action_probability in enumerate(policy[state]):
                             # Check how good next state will be
                             for state_probability, next_state, reward, terminated in environment.P[state][action]:
                                  # Calculate the expected value
                                  v += action_probability * state_probability * (reward + discount_factor * V[next_state])
                       
                       # Calculate the absolute change of value function
                       delta = max(delta, np.abs(V[state] - v))
                       # Update value function
                       V[state] = v
                evaluation_iterations += 1
                
                # Terminate if value change is insignificant
                if delta < theta:
                        print(f'Policy evaluated in {evaluation_iterations} iterations.')
                        return V

In [35]:
def one_step_lookahead(environment, state, V, discount_factor):
        action_values = np.zeros(environment.action_space.n)
        for action in range(environment.action_space.n):
                for probability, next_state, reward, terminated in environment.P[state][action]:
                        action_values[action] += probability * (reward + discount_factor * V[next_state])
        return action_values

In [37]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
        # Start with a random policy
        #num states x num actions / num actions
        policy = np.ones([environment.observation_space.n, environment.action_space.n]) / environment.action_space.n
        
        
        # Initialize counter of evaluated policies
        evaluated_policies = 1
        # Repeat until convergence or critical number of iterations reached
        for i in range(int(max_iterations)):
                stable_policy = True
                # Evaluate current policy
                V = policy_evaluation(policy, environment, discount_factor=discount_factor)
                # Go through each state and try to improve actions that were taken (policy Improvement)
                for state in range(environment.observation_space.n):
                        # Choose the best action in a current state under current policy
                        current_action = np.argmax(policy[state])
                        # Look one step ahead and evaluate if current action is optimal
                        # We will try every possible action in a current state
                        action_value = one_step_lookahead(environment, state, V, discount_factor)
                        # Select a better action
                        best_action = np.argmax(action_value)
                        # If action didn't change
                        if current_action != best_action:
                                stable_policy = True
                                # Greedy policy update
                                policy[state] = np.eye(environment.action_space.n)[best_action]
                evaluated_policies += 1
                # If the algorithm converged and policy is not changing anymore, then return final policy and value function
                if stable_policy:
                        print(f'Evaluated {evaluated_policies} policies.')
                        return policy, V

In [38]:
def play_episodes(environment, n_episodes, policy):
        wins = 0
        total_reward = 0
        for episode in range(n_episodes):
                terminated = False
                state = environment.reset()
                while not terminated:
                        # Select best action to perform in a current state
                        action = np.argmax(policy[state])
                        # Perform an action an observe how environment acted in response
                        next_state, reward, terminated, info = environment.step(action)
                        # Summarize total reward
                        total_reward += reward
                        # Update current state
                        state = next_state
                        # Calculate number of wins over episodes
                        if terminated and reward == 1.0:
                                wins += 1
        average_reward = total_reward / n_episodes
        return wins, total_reward, average_reward

# Number of episodes to play
n_episodes = 10000
# Functions to find best policy
solvers = [
    ('Policy Iteration', policy_iteration),
]
for iteration_name, iteration_func in solvers:
        # Load a Frozen Lake environment
        environment = gym.make('FrozenLake-v1')
        # Search for an optimal policy using policy iteration
        policy, V = iteration_func(environment.env)
        # Apply best policy to the real environment
        wins, total_reward, average_reward = play_episodes(environment, n_episodes, policy)
        print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
        print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward} \n\n')

Policy evaluated in 66 iterations.
Evaluated 2 policies.
Policy Iteration :: number of wins over 10000 episodes = 7259
Policy Iteration :: average reward over 10000 episodes = 0.7259 




In [20]:
environment.class_

4